In [ ]:
import numpy as np
import scipy.io

# GPS PRN Extraction
codesFile = scipy.io.loadmat("Dataset2.mat")
codes = np.array(codesFile['sim_doppler_l'])

# Function to get modified PRN for some sampling frequency & delay
def modPRN(PRN, n, tau):
    return np.repeat(np.roll(PRN, -tau), n)

# Serial search function
def serialSearch(I, Q, knownPRNs, Fs, wds):
    rcvdSignal = np.array(I + Q * 1j)
    n = int(Fs / 1.023e6)
    t = np.linspace(0, 1, n * 1023) * 1e-3
    totalSatellites = knownPRNs.shape[1]
    corr = np.zeros((totalSatellites, len(wds), 1023 * n))
    
    for x in range(totalSatellites):
        for y in range(len(wds)):
            for z in range(1023 * n):
                generatedSignal = modPRN(knownPRNs[:, x], n, z) * np.exp(-1j * wds[y] * t)
                corr[x, y, z] = np.abs(np.sum(generatedSignal * rcvdSignal))
    
    return corr

# Example usage
wds = np.linspace(-20000, 20000, 41) * 2 * np.pi
t = np.linspace(0, 1, 1023) * 1e-3

sirPRNI = modPRN(codes[:, 20], 1, 82) * np.cos(2 * np.pi * 675 * t) + np.random.normal(0, 4, 1023)
sirPRNQ = modPRN(codes[:, 20], 1, 82) * np.sin(2 * np.pi * 675 * t) + np.random.normal(0, 4, 1023)

corr = serialSearch(sirPRNI, sirPRNQ, codes, 1.023e6, wds)
satNum, wdstar, taustar = np.unravel_index(corr.argmax(), corr.shape)

print("Satellite Number:", satNum)
print("Carrier Frequency:", wds[wdstar] / (2 * np.pi))
print("Code Phase:", taustar)
